In [18]:
from interpreter import interpret

In [19]:
code = 'f := Lx.(x + 1)\ng := Lx.(x + 2)\ng(f(1))'
interpret(code)

'>>> >>> 4\n>>> '

In [20]:
#add(x)(y)
code = '''
add := Lx.Ly.(x + y)
add(1)(add(2)(3))
'''
print(interpret(code))


>>> 6
>>> 


In [21]:
code = '''
TRUE := Lx.Ly.x
FALSE := Lx.Ly.y
IF := Lc.La.Lb.(c a b)
IF TRUE 1 2
IF FALSE 1 2
'''
print(interpret(code))

>>> >>> >>> 1
>>> 2
>>> 


In [22]:
code = '''
TRUE := Lx.Ly.x
FALSE := Lx.Ly.y
IF := Lc.La.Lb.(c a b)
IF TRUE 1 2
'''
print(interpret(code))

>>> >>> >>> 1
>>> 
